# Une prise en main rapide de Jupyter / Julia / JuMP

## Qu'est-ce qu'un Jupyter notebook ?

Un Jupyter notebook est un document qui contient 
+ du texte 
  - que l'on peut formatter à l'aide de Markdown
  - qui peut contenir des maths à l'aide de $\LaTeX$
+ du code
  - avec lequel on peut intéragir en ligne
  
Un notebook est une succession de cellule, chacune pouvant être soit du code, soit du texte.
Quelques astuces :
+ double-clicker pour rentrer dans une cellule
+ M / Y pour changer le type de cellule
+ Ctrl-enter pour éxecuter la cellule
+ shift-enter pour éxecuter la cellule et passer à la suivante
+ Alt-enter pour éxecuter la cellule et en ajouter une nouvelle

Vous pouvez télécharger le fichier .ipynb via l'onglet "file" en haut à gauche. Vous pouvez aussi télécharger un pdf.

## Qu'est-ce que Julia ?

Julia est un langage de programmation, comparable à Python. C'est un langage récent, développé pour le calcul scientifique. 

Quelques éléments intéressant :
+ langage open-source
+ langage compilé "Just-in-time"
+ langage disposant d'un terminal (comme python)
+ ...

Faisons nos premiers pas avec Julia. Exécuter les cellules suivantes (shift-enter), n'hésitez pas à modifier pour prendre en main :

In [ ]:
1+1

In [ ]:
a = [0 5 10 15]
a[1]

In [ ]:
sum(a)

In [ ]:
sum(x^2 for x in a)

In [ ]:
length(a)

In [ ]:
exp.(a) .- a #to apply an operation or function componentwise just add .  

In [ ]:
@doc exp # to get documentation on a function

In [ ]:
for i = 1:5
    println("itération ",i)
end

In [ ]:
function factorielle(n)
    if n == 0
        return 1
    end
    res = 1
    for i=1:n
        res = res * i
    end
    return res
end

In [ ]:
factorielle(5)

In [ ]:
round(1.9453;sigdigits=2)

In [ ]:
rand(3)

Further info can be found [here](https://learnxinyminutes.com/docs/julia/) or in the [documentation](https://docs.julialang.org/en/v1/)



## Qu'est-ce que JuMP ?

JuMP est l'un des packages phare de Julia.

Il s'agit d'un package de modélisation, qui permet d'écrire un problème d'optimisation de manière simple puis de demander à un Solver de le résoudre.

Nous allons maintenant faire nos premiers pas avec JuMP.

Plus d'information sur http://www.juliaopt.org/JuMP.jl/v0.20.0/quickstart/ 

### Mise en place

Avant toute chose il faut installer puis appeler les bibliothèques utiles.


In [ ]:
import Pkg;
Pkg.add("GLPK")
Pkg.add("Ipopt")
Pkg.add("JuMP")

On peut faire la liste des packages installés et leur version en executant la commande suivante.

In [ ]:
Pkg.status()

Nous allons maintenant dire que nous souhaitons utiliser ces packages (comparable à "from packet import *" en python)

In [ ]:
using JuMP, Ipopt, GLPK


## Construction d'un premier problème linéaire

Nous souhaitons résoudre le problème linéaire suivant
$$ \begin{align*} 
\min_{x,y} \quad & 2x+3y \\
s.c. \quad & x+y \geq 1 \\
& x \geq 0, y\geq 0 \\
\end{align*}$$

Commençons par construire le problème

In [ ]:
OPTIMIZER = GLPK.Optimizer              # On définit un optimizer
m = Model(with_optimizer(OPTIMIZER))       # On construit un problème d'optimisation

@variable(m,x>=0)                       # x est une variable réelle positive de m
@variable(m,y>=0)                       # y est une variable réelle positive de m

### Remarque les fonctions @variable / @objective / @constraint sont des fonctions spécifiques (des macros) 
# qui autorise de donner un argument comme 2*x+3*y sans qu'il soit évalué. 
# Ce n'est pas un comportement générique des fonctions julia.

@objective(m,Min, 2*x+3*y)              # l'objectif de m est de Minimiser 2*x+3*y

@constraint(m,x+y >= 1 )                # m a pour contrainte x+y <=1

On peut vérifier que m est bien ce que l'on souhaite

In [ ]:
print(m)

On peut également résoudre m

In [ ]:
optimize!(m)
println(termination_status(m))
println(primal_status(m))
println(dual_status(m))

Et si on souhaite connaître la valeur optimale du problème ou des solutions optimales on peut les avoir de la manière suivante

In [ ]:
println(JuMP.objective_value(m))
println("x = ",JuMP.value(x))
println("y = ",JuMP.value(y))

### Un second problème linéaire

Nous allons maintenant construire un problème linéaire plus complexe.
$$
\begin{align*}
\min_{x\in R^n} \quad & \sum_{i=1}^n c_i x_i \\
s.c. \quad & \sum_{i=1}^n x_i \geq n \\
&  -1 \leq x_i \leq 2 & \forall i
\end{align*}
$$

In [ ]:
m2 = Model(with_optimizer(OPTIMIZER))
n = 10                                    # on choisit n = 10, mais vous pouvez le modifier
c = rand(n)                               # c est choisi ici de manière aléatoire                                              

@variable(m2, -1<= x[1:n] <= 2)           # x est une variable de m2 contenant n éléments x[1], x[2],...,x[n] tous compris entre -1 et 2

@objective(m2,Min, sum(c[i]*x[i] for i=1:n) )

@constraint(m2,sum(x[i] for i=1:n) >= n)
                        
print(m2)            

In [ ]:
optimize!(m2)
println(termination_status(m2))

In [ ]:
value.(x)

Ajoutons maintenant une série de contraintes de la forme
$$ x_i + x_{i+1} \leq 1, \qquad \forall i \in 2, \dots, n-1$$

In [ ]:
for i = 1 : n-1
    @constraint(m2, x[i]+x[i+1] <= 2)
end

In [ ]:
optimize!(m2)
value.(x)

### Un problème non-linéaire

Nous allons terminer avec un problème non-linéaire simple.

$$
\begin{align*}
\min_{x \in R^{n.m}} \quad & \sum_{i,j} x_{i,j}^2 \\
s.c. \quad & \sum_{i=1}^n x_{i,j} = 1 & \forall j \in [m] 
\end{align*}
$$

In [ ]:
using Ipopt

In [ ]:
OPTIMIZER_NL = Ipopt.Optimizer

m3 = Model(with_optimizer(OPTIMIZER_NL))

N,M = 5,7 

@variable(m3, x[i=1:N, j=1:M])

@objective(m3, Max, sum(x[i,j]^2 for i=1:N, j=1:M))

for j=1:M
    @constraint(m3, sum(x[i,j] for i =1:N)==1)
end

In [ ]:
print(m3)

In [ ]:
optimize!(m3)
println(objective_value(m3))

In [ ]:
value.(x)

Tested Version :
- JuMP 0.20 / Julia 1.0.5
- JuMP 0.20 / Julia 1.2.0
- JuMP 0.20 / Julia 1.3.0-rc3